# Monte Carlo Estimation of Pi
How do we know the value of pi? We have to compute it. One way to compute it is through random sampling. Image a 1x1 square with a cirle enscribed in it. If we randomly place dots in the sqaure, the ratio of the number of dots in the sqaure $N_s$ to the number of dots in the circle $N_c$ will estimate the ratio of the areas of the two shapes $ \frac{N_s}{N_c} = \frac{A_s}{A_c}$. Since $A_c = 1^2 = 1$ and $A_s = \pi 1^2 = \pi$, $\pi = \frac{N_s}{N_c}$

### Random Sample Generator: Classical vs Quantum

Can we see performance difference between a classical random number generator 